In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

# Explicit Waits 사용을 위한 라이브러리
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.common.exceptions import NoSuchElementException

# 크롬 드라이버 자동 업데이트
from webdriver_manager.chrome import ChromeDriverManager

import time

In [2]:
# 브라우저 꺼짐 방지
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)

In [3]:
# 불필요한 에러 메시지 없애기
chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])

In [4]:
service = Service(executable_path=ChromeDriverManager().install())
service

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - About to download new driver from https://chromedriver.storage.googleapis.com/102.0.5005.61/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\Administrator\.wdm\drivers\chromedriver\win32\102.0.5005.61]


In [5]:
driver = webdriver.Chrome(service=service, options=chrome_options)
driver

<selenium.webdriver.chrome.webdriver.WebDriver (session="2752e2a5f864f9e559c806d939f3229c")>

In [6]:
# 웹페이지 해당 주소 이동
driver.implicitly_wait(5)  # 웹페이지가 로딩 될때까지 5초 기다림
driver.maximize_window()  # 화면 최대화

# 20대 대통령 선거 스크래핑

In [7]:
# 시작. 페이지로 출발
driver.get('http://info.nec.go.kr/')

In [8]:
driver.switch_to.default_content()

In [9]:
driver.switch_to.frame('main')

In [10]:
driver.find_element(By.XPATH, '//*[@id="header"]/ul/li[1]/a/span[text() = "최근선거"]').click()
driver.implicitly_wait(5)

In [11]:
driver.find_element(By.XPATH, '//*[@id="menu"]/ul/li[2]/a[text() = "제20대 대통령선거"]').click()
driver.implicitly_wait(5)

In [12]:
driver.find_element(By.XPATH, '//*[@id="VC"]/span[text() = "투·개표"]').click()
driver.implicitly_wait(5)

### 개표단위별 개표결과 데이터프레임 처리 및 스크래핑한 자료 검증

- 콤보박스에서 조회할 목록(시도명, 구시군명)을 가져오고 스크랩한 데이터의 조회결과 내용과 같은지 비교한다.
- 매칭이 되지 않을 경우 프로그램 중단

In [13]:
# 개표단위별 개표결과
def get_data2(city, town):
    html = driver.page_source

    df1 = pd.read_html(html, header=1)
    df2 = df1[0].copy()
    
    result = driver.find_element(By.XPATH, '//*[@id="contentarea"]/div[2]/div[2]/div[1]/p[1][@class="selectCondition"]').text
    result = result.strip().replace('[', '').replace(']', '').replace('\n', ' ')
    
    if city in result:
        if town in result:
            df2['조회결과'] = result
        else:
            print(f'에러 - 조회결과에 문제가 있습니다.\n요청 : {city} {town}\n결과 : {result}')
            print('프로그램을 중단합니다.')
            sys.exit()
    else:
        print(f'에러 - 조회결과에 문제가 있습니다\n요청 : {city}\n결과 : {result}')
        print('프로그램을 중단합니다.')
        sys.exit()

    return df2

In [14]:
# 개표단위별 개표결과

driver.find_element(By.XPATH, '//*[@id="gnb"]/div[4]/ul/li[4]/a[text() = "개표단위별 개표결과"]').click()
time.sleep(1)

driver.find_element(By.XPATH, '//*[@id="electionId1"][text() = "대통령선거"]').click()
time.sleep(1)

df_20 = pd.DataFrame()

combo_box_1 = driver.find_element(By.XPATH, '//*[@id="cityCode"]')
cityCodes = [option.text for option in combo_box_1.find_elements(By.TAG_NAME, "option")]
cityCodes.pop(0) # 선택 지우기

for cityCode in cityCodes:
    driver.find_element(By.XPATH, f'//*[@id="cityCode"]').send_keys(cityCode)
    time.sleep(1)
    
    combo_box_2 = driver.find_element(By.XPATH, '//*[@id="townCode"]')
    townCodes = [option.text for option in combo_box_2.find_elements(By.TAG_NAME, "option")]
    townCodes.pop(0)    
    
    for townCode in townCodes:
        # print(cityCode, townCode)
        
        driver.find_element(By.XPATH, f'//*[@id="townCode"]').send_keys(townCode)
        time.sleep(1)
        
        driver.find_element(By.XPATH, '//*[@id="spanSubmit"]/input[@title="검색"]').click()
        driver.implicitly_wait(5)
        
        df_20 = pd.concat([df_20, get_data2(cityCode, townCode)], ignore_index=True)
        time.sleep(1)

## 20대 대선 데이터프레임_원본 (df_20)

In [15]:
# 선거구분, 시도명, 구시군명 열 분리

df_20['선거구분'] = df_20['조회결과'].str.split().str[0]
df_20['시도명'] = df_20['조회결과'].str.split().str[1]
df_20['구시군명'] = df_20['조회결과'].str.split().str[2]
df_20 = df_20[['선거구분', '시도명', '구시군명', '읍면동명', '투표구명', '선거인수', '투표수',
               '더불어민주당이재명', '국민의힘윤석열', '정의당심상정', '기본소득당오준호',
               '국가혁명당허경영', '노동당이백윤', '새누리당옥은호', '신자유민주연합김경재', '우리공화당조원진',
               '진보당김재연', '통일한국당이경희', '한류연합당김민찬', '계', '무효투표수', '기권수', '조회결과']]
df_20

,선거구분,시도명,구시군명,읍면동명,투표구명,선거인수,투표수,더불어민주당이재명,국민의힘윤석열,정의당심상정,...,새누리당옥은호,신자유민주연합김경재,우리공화당조원진,진보당김재연,통일한국당이경희,한류연합당김민찬,계,무효투표수,기권수,조회결과
0,대통령선거,서울특별시,종로구,합계,NaN,129968,100629,46130,49172,3115,...,9,50,82,105,20,30,99366,1263,29339,대통령선거 서울특별시 종로구
1,대통령선거,서울특별시,종로구,거소·선상투표,NaN,217,200,82,85,4,...,0,0,0,2,0,0,177,23,17,대통령선거 서울특별시 종로구
2,대통령선거,서울특별시,종로구,관외사전투표,NaN,13132,13092,6416,5972,434,...,1,5,3,22,2,4,12970,122,40,대통령선거 서울특별시 종로구
3,대통령선거,서울특별시,종로구,재외투표,NaN,2077,1480,739,541,56,...,0,0,0,1,0,0,1348,132,597,대통령선거 서울특별시 종로구
4,대통령선거,서울특별시,종로구,재외투표(공관),NaN,0,188,83,79,6,...,0,0,0,0,0,0,168,20,-188,대통령선거 서울특별시 종로구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22730,대통령선거,제주특별자치도,서귀포시,NaN,관내사전투표,881,881,435,406,20,...,0,0,0,3,1,1,878,3,0,대통령선거 제주특별자치도 서귀포시
22731,대통령선거,제주특별자치도,서귀포시,NaN,예래동제1투,701,421,180,204,13,...,0,2,0,0,0,1,410,11,280,대통령선거 제주특별자치도 서귀포시
22732,대통령선거,제주특별자치도,서귀포시,NaN,예래동제2투,1113,569,269,267,16,...,0,0,1,1,0,0,558,11,544,대통령선거 제주특별자치도 서귀포시
22733,대통령선거,제주특별자치도,서귀포시,NaN,예래동제3투,329,225,126,91,7,...,0,0,0,0,0,0,225,0,104,대통령선거 제주특별자치도 서귀포시


### 20대 대선 구시군 합계 (df_20_sum)

In [16]:
df_20_sum = df_20[df_20['읍면동명'] == '합계'].copy()
df_20_sum

,선거구분,시도명,구시군명,읍면동명,투표구명,선거인수,투표수,더불어민주당이재명,국민의힘윤석열,정의당심상정,...,새누리당옥은호,신자유민주연합김경재,우리공화당조원진,진보당김재연,통일한국당이경희,한류연합당김민찬,계,무효투표수,기권수,조회결과
0,대통령선거,서울특별시,종로구,합계,NaN,129968,100629,46130,49172,3115,...,9,50,82,105,20,30,99366,1263,29339,대통령선거 서울특별시 종로구
83,대통령선거,서울특별시,중구,합계,NaN,111448,84998,38244,42906,2310,...,10,17,67,57,17,28,84184,814,26450,대통령선거 서울특별시 중구
162,대통령선거,서울특별시,용산구,합계,NaN,199077,152068,60063,85047,4374,...,24,56,86,100,27,46,150682,1386,47009,대통령선거 서울특별시 용산구
259,대통령선거,서울특별시,성동구,합계,NaN,252087,197240,84411,103880,5365,...,20,47,122,152,34,65,195238,2002,54847,대통령선거 서울특별시 성동구
369,대통령선거,서울특별시,광진구,합계,NaN,303582,235471,109922,113733,7072,...,36,62,180,172,38,80,232918,2553,68111,대통령선거 서울특별시 광진구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22255,대통령선거,경상남도,산청군,합계,NaN,31645,24913,7125,16607,464,...,4,9,34,82,42,28,24646,267,6732,대통령선거 경상남도 산청군
22298,대통령선거,경상남도,거창군,합계,NaN,53049,41399,11963,27254,895,...,18,12,42,92,62,52,40945,454,11650,대통령선거 경상남도 거창군
22348,대통령선거,경상남도,합천군,합계,NaN,39768,31270,6911,22742,511,...,9,16,62,96,68,52,30832,438,8498,대통령선거 경상남도 합천군
22409,대통령선거,제주특별자치도,제주시,합계,NaN,408552,296826,157695,122084,10007,...,46,80,193,395,159,197,293649,3177,111726,대통령선거 제주특별자치도 제주시


In [17]:
# XlsxWriter를 엔진으로 사용하여 Pandas Excel Writer를 작성
writer = pd.ExcelWriter('20대대선.xlsx', engine='xlsxwriter')

# 데이터 프레임을 각각 워크시트에 작성
df_20.to_excel(writer, sheet_name='20대_대선', index=False)
df_20_sum.to_excel(writer, sheet_name='20대_대선_합계', index=False)

# Pandas Excel Writer를 닫고 Excel 파일 저장
writer.save()

In [18]:
driver.quit()